<!-- <a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>
 -->
 
<!-- # Released under  -->

Copyright (c) 2013 Mark Otto. Copyright (c) 2017 Andrew Fong. MIT License

<h1><center>Xây dựng mô hình Random Forest cho bài toán phân loại</center></h1>

 - Trong bài thực hành này, chúng ta sử dụng mô hình Random Forest để phân loại bệnh nhân vào các loại thuốc khác nhau. Mô hình sau khi huấn luyện được sử dụng để dự đoán nhãn cho bệnh nhân mới từ đó có thể tìm được thuốc thích hợp
<!-- In this lab exercise, you will learn a popular machine learning algorithm, Decision Tree. You will use this classification algorithm to build a model from historical data of patients, and their response to different medications. Then you use the trained decision tree to predict the class of a unknown patient, or to find a proper drug for a new patient. -->

<h1>Nội dung thực hành</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#about_dataset">Giới thiệu về dữ liệu</a></li>
        <li><a href="#downloading_data">Tải dữ liệu</a></li>
        <li><a href="#pre-processing">Tiền xử lý dữ liệt</a></li>
        <li><a href="#setting_up_tree">Cài đặt mô hình Random Forest</a></li>
        <li><a href="#modeling">Huấn luyện</a></li>
        <li><a href="#prediction">Dự đoán</a></li>
        <li><a href="#evaluation">Đánh giá mô hình</a></li>
        <li><a href="#evaluation">Đánh giá ảnh hưởng của các thuộc tính với nhãn</a></li>
    </ol>
</div>
<br>
<hr>

Import các thư viện sau:
<ul>
    <li> <b>numpy (as np)</b> </li>
    <li> <b>pandas</b> </li>
    <li> <b>RandomForestClassifier</b> from <b>sklearn.ensemble</b> </li>
</ul>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

<div id="Giới thiệu về dữ liệu">
    <h2>Giới thiệu về dữ liệu</h2>
    Giả sử bạn là một người nghiên cứu về dữ liệu y học, bạn đã thu thập được một tập dữ liệu các bệnh nhân của cùng một loại bệnh, trong khi điều trị mỗi bệnh nhân phản ứng với 1 trong 5 loại thuốc A, B, c, x và y.
    <br>
    <br>
    Công việc cần làm là xây dựng một mô hình nhằm tìm ra loại thuốc phù hợp cho bênh nhân mới bị mắc loại bênh trên. Các trường dữ liệu gồm gồm có: Age, Sex, Blood Pressur, Cholesterol, Na_to_K, Drug
    <br>
    <br>
    Chúng ta sử dụng phần dữ liệu huấn luyện để xây dựng mô hình Random Forest và đánh giá mô hình này trên tập đánh giá, sau đó sử dụng mô hình để suy diễn trên bệnh nhân mới
</div>


<div id="downloading_data"> 
    <h2>Load dữ liệu từ file</h2>
<!--     Sử dụng wget để tải dữ liệu từ IBM Object Storage. -->
</div>

Đọc dữ liệu vào pandas dataframe

In [ ]:
my_data = pd.read_csv("data/drug200.csv", delimiter=",")
my_data[0:5]

Kích thước của dữ liệu

In [ ]:
my_data.shape

<div href="pre-processing">
    <h2>Tiền xử lý dữ liệu</h2>
</div>

<b>my_data</b> là  Drug.csv sau khi đọc vào pandas dataframe, chúng ta định nghĩa các tham số sau: <br>

<ul>
    <li> <b> X </b> as the <b> Ma trận thuộc tính </b> (data of my_data) 
    <li> <b> y </b> as the <b> Vector thuốc phản hồi (target) </b> </li>
</ul>

In [ ]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

Một vài trường trong dữ liệu có dạng categorical như __Sex__ or __BP__. Mô hình Random Forest chỉ xử lý với dữ liệu số thực do đó chúng ta cần chuyển các thuộc tính trên về dạng số thực tương ứng. Sử dụng  __skearn.preprocessing__ để chuyển dữ liệu dạng categorical về số thực

In [ ]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

X[0:5]

Dữ liệu target về loại thuốc phản ứng

In [ ]:
y = my_data["Drug"]
y[0:5]

<hr>

<div id="setting_up_tree">
    <h2>Cài đặt mô hình Random Forest</h2>
    Chúng ta chia dữ liệu thành hai phần <b>train</b> và <b>test</b> để huấn luyện và đánh giá mô hình <b>Random Forest</b>. Import <b>train_test_split</b> từ <b>sklearn.cross_validation</b>.
</div>

In [ ]:
from sklearn.model_selection import train_test_split

Hàm <b> train_test_split </b> trả về 4 giá trị khác nhau. Chúng ta đặt tên như sau:<br>
X_trainset, X_testset, y_trainset, y_testset <br> <br>
Hàm <b> train_test_split </b> cần các tham số đầu vào: <br>
X, y, test_size=0.3, and random_state=3. <br> <br>
<b>X</b> and <b>y</b> là các ma trận và vector cần chia, <b>test_size</b> là tỉ lệ tập test,và <b>random_state</b> để loại bỏ tính ngẫu nhiên.

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

<h3>Thực hành</h3>
Print kích thước X_trainset and y_trainset. Đảm bảo số chiều tương ứng với nhau

In [ ]:
# your code



Print kích thước X_testset and y_testset. Đảm bảo số chiều tương ứng với nhau

In [ ]:
# your code



<hr>

<div id="modeling">
    <h2>Huấn luyện</h2>
    Trước tiên chúng ta cần tạo một đối tượng <b>RandomForestClassifier</b> tên <b>drugTree</b>.<br>
    Khi khởi tạo cần chỉ rõ số cây tương ứng
</div>

In [ ]:
drugTree = RandomForestClassifier(n_estimators=100)
drugTree

Huấn luyện mô hình trên tập <b> X_trainset </b> và vector nhãn tương ứng <b> y_trainset </b>

In [ ]:
drugTree.fit(X_trainset,y_trainset)

<hr>

<div id="prediction">
    <h2>Dự đoán</h2>
    Thực hiện các <b>dự đoán</b> trên tập dữ liệu test và lưu kết quả vào biến <b>predTree</b>.
</div>

In [ ]:
predTree = drugTree.predict(X_testset)

So sánh kết quả dữ đoán <b>predTree</b> và nhãn thật tương ứng <b>y_testset</b>

In [ ]:
print('Nhãn phán đoán')
print (predTree [0:5])
print('Nhãn đúng')
print(list(y_testset)[0:5])

<hr>

<div id="evaluation">
    <h2>Đánh giá mô hình</h2>
    Import <b>metrics</b> từ sklearn and kiểm tra độ chính xác của mô hình trên tập test
</div>

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

<hr>

<div id="evaluation">
    <h2>Đánh giá ảnh hưởng của các thuộc tính với nhãn</h2>
        Đánh giá độ ảnh hưởng của thuộc tính với nhãn là một kỹ thuật để gán điểm số cho các thuộc tính đầu vào của mô hình nhằm cho biết tầm quan trọng của thuộc tính đó khi đưa ra dự đoán.
    <br>
    <br>
    Các điểm số có thể được sử dụng để:
    <ul>
    <li> Hiểu về dữ liệu
    <li> Hiểu về dữ liệu
    <li> Giảm số chiều của dữ liệu bằng việc loại đi các thuộc tính không quan trọng </li>
</ul>
</div>


In [ ]:
print(list(my_data.columns)[0:-1])
print(list(drugTree.feature_importances_))